# Step-by-step demo walk through

The testing demos are intended to be done in a code editor/IDE, and the tests run from the command line after each step. Below each of the steps are laid out to guide the tutorial leader. I prefer to print this out so I can refer to it while I live code each step over a projector along with the attendees.

---

# TDD Demo

Both the TDD and the Fixtures demo use the same file `demo_tdd_intro.py`. Run the tests after each step of each task below, on the command line using: `pytest demo_tdd_intro.py`

In [1]:
# initial code in demo_tdd_intro.py

def backwards_allcaps(text):
    return text[::-1].upper()


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'

### TDD task 1: whitespace should be removed from any input text

In [2]:
def backwards_allcaps(text):
    return text[::-1].replace(' ', '').upper()            # step 2


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'


def test_letters_only():
    assert backwards_allcaps('salt lake') == 'EKALTLAS'    # step 1

### TDD task 2: punctuation should also be removed

In [3]:
import string                                                          # step 2


def backwards_allcaps(text):
    for char in string.punctuation:                                    # step 2
        text = text.replace(char, '')                                  # step 2
    return text[::-1].replace(' ', '').upper()


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'


def test_letters_only():
    assert backwards_allcaps('salt lake') == 'EKALTLAS'
    assert backwards_allcaps('github.com is rad!') == 'DARSIMOCBUHTIG'  # step 1

### TDD task 3: passing an empty string should raise an error

In [4]:
import pytest                                                # step 2
import string


def backwards_allcaps(text):
    if len(text) == 0:                                       # step 2
        raise AttributeError('String must contain letters')  # step 2
    for char in string.punctuation:
        text = text.replace(char, '')
    return text[::-1].replace(' ', '').upper()


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'


def test_letters_only():
    assert backwards_allcaps('salt lake') == 'EKALTLAS'
    assert backwards_allcaps('github.com is rad!') == 'DARSIMOCBUHTIG'


def test_bad_string():                                       # step 1
    with pytest.raises(AttributeError):
        backwards_allcaps('')

### TDD task 3b: passing a string without letters should raise an error

In [5]:
import pytest
import string


def backwards_allcaps(text):
    text = text.replace(' ', '')                             # step 2 - move section
    for char in string.punctuation:
        text = text.replace(char, '')
    if len(text) == 0:                                       # step 2 - move section
        raise AttributeError('String must contain letters')
    return text[::-1].upper()


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'


def test_letters_only():
    assert backwards_allcaps('salt lake') == 'EKALTLAS'
    assert backwards_allcaps('github.com is rad!') == 'DARSIMOCBUHTIG'


def test_bad_string():
    with pytest.raises(AttributeError):
        backwards_allcaps('')
    with pytest.raises(AttributeError):                     # step 1
        backwards_allcaps(';')                              # step 1
    with pytest.raises(AttributeError):                     # step 1
        backwards_allcaps(' ')                              # step 1

# Fixtures Demo

Most of the tests we have written use the same pattern of asserting that our function run on some input gives us an expected output. Instead of reusing this same assert pattern repeatedly we would like a way to avoid duplicating code.

Additionally, each assert statement can be thought of as a unique test, but with the current set up pytest treats each test function as a unique test instead, which means that if the first assert statement in a test function fails, the assertions after it *are not run*.

As an example, run the tests again with the typo below introduced. Try using the verbose flag `pytest -v demo_tdd_intro.py`.

In [6]:
def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOTYP'   # NOTICE if this line fails
    assert backwards_allcaps('meetup') == 'PUTEEM'  # then this line won't get run

### Parametrize the data test cases with a pytest fixture

Using fixtures abstracts the data test cases away from the actual test itself, and allows all assertions to be checked and treated as separate tests.

In [7]:
import pytest
import string


def backwards_allcaps(text):
    text = text.replace(' ', '')
    for char in string.punctuation:
        text = text.replace(char, '')
    if len(text) == 0:
        raise AttributeError('String must contain letters')
    return text[::-1].upper()


@pytest.fixture(params=[
    {'input': 'python', 'output': 'NOHTYP'},
    {'input': 'meetup', 'output': 'PUTEEM'},
    {'input': 'salt lake', 'output': 'EKALTLAS'},
    {'input': 'github.com is rad!', 'output': 'DARSIMOCBUHTIG'}])
def test_data(request):
    return request.param


def test_backwards_allcaps(test_data):
    assert backwards_allcaps(test_data['input']) == test_data['output']


def test_bad_string():
    with pytest.raises(AttributeError):
        backwards_allcaps('')
    with pytest.raises(AttributeError):
        backwards_allcaps(';')
    with pytest.raises(AttributeError):
        backwards_allcaps(' ')

### Create a fixture for the inputs that should raise errors

In [8]:
import pytest
import string


def backwards_allcaps(text):
    text = text.replace(' ', '')
    for char in string.punctuation:
        text = text.replace(char, '')
    if len(text) == 0:
        raise AttributeError('String must contain letters')
    return text[::-1].upper()


@pytest.fixture(params=[
    {'input': 'python', 'output': 'NOHTYP'},
    {'input': 'meetup', 'output': 'PUTEEM'},
    {'input': 'salt lake', 'output': 'EKALTLAS'},
    {'input': 'github.com is rad!', 'output': 'DARSIMOCBUHTIG'}])
def test_data(request):
    return request.param


@pytest.fixture(params=['', ';', ' '])
def bad_input(request):
    return request.param


def test_backwards_allcaps(test_data):
    assert backwards_allcaps(test_data['input']) == test_data['output']


def test_bad_string(bad_input):
    with pytest.raises(AttributeError):
        backwards_allcaps(bad_input)

### All test cases will be run, even if a prior one fails

For example, add the parameter 'abc' to the `bad_input` fixture and run the tests with the verbose flag. This test will fail, but the tests afterwards are still run!

In [9]:
@pytest.fixture(params=['', 'abc', ';', ' '])
def bad_input(request):
    return request.param

---

# Legacy Code Demo

This demo uses the python file `demo_legacy_code.py`.